In [13]:
import pymongo
from pymongo import MongoClient
from dotenv import load_dotenv
import os

load_dotenv()

True

In [14]:
connection_string = os.getenv("DB_CONNECTION_SECRET")

In [15]:
client = MongoClient(
        connection_string,
        connectTimeoutMS=5000,  # 5 seconds
        serverSelectionTimeoutMS=5000  # 5 seconds
    )

In [16]:
db = client.get_database("devTinder")

In [22]:
from langchain_mongodb.agent_toolkit import MongoDBDatabase, MongoDBDatabaseToolkit
from langchain_groq import ChatGroq

# Set up the MongoDB database
db = MongoDBDatabase.from_connection_string(
    os.getenv("DB_CONNECTION_SECRET"),
    database="devTinder"
)

llm = ChatGroq(model="llama3-8b-8192", api_key=os.getenv("GROQ_API_KEY"))

# Create the toolkit with the database
toolkit = MongoDBDatabaseToolkit(db=db, llm=llm)

# Get the tools from the toolkit
tools = toolkit.get_tools()

print("Available tools:")
for tool in tools:
    print(tool.name)

Available tools:
mongodb_query
mongodb_schema
mongodb_list_collections
mongodb_query_checker


In [34]:
db.get_usable_collection_names()

['chats', 'connectionrequests', 'payments', 'test_collection', 'users']

In [35]:
llm.invoke("Hello, how are you?")

AIMessage(content="I'm just a language model, I don't have feelings or emotions like humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have! It's great to chat with you. How can I help you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 16, 'total_tokens': 69, 'completion_time': 0.044166667, 'prompt_time': 0.002382462, 'queue_time': 0.233015747, 'total_time': 0.046549129}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-820baea0-1416-474d-9648-f4d93bcc623e-0', usage_metadata={'input_tokens': 16, 'output_tokens': 53, 'total_tokens': 69})

In [36]:
for tool in tools:
    # Check if the tool's name is "mongodb_list_collections"
    if tool.name == "mongodb_list_collections":
        list_collections_tool = tool
        break

In [37]:
for tool in tools:
    # Check if the tool's name is "mongodb_schema"
    if tool.name == "mongodb_schema":
        # If found, assign it to get_schema_tool
        get_schema_tool = tool
        break 

In [38]:
llm_to_get_schema=llm.bind_tools([get_schema_tool])
llm_to_get_schema

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1213d4cd0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1213d6e90>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'mongodb_schema', 'description': 'Input to this tool is a comma-separated list of collections, output is the schema and sample rows for those collections. Be sure that the collectionss actually exist by calling mongodb_list_collections first! Example Input: collection1, collection2, collection3', 'parameters': {'properties': {'collection_names': {'description': "A comma-separated list of the collection names for which to return the schema. Example input: 'collection1, collection2, collection3'", 'type': 'string'}}, 'required': ['collection_names'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [100]:
from langchain.tools import tool

# The @tool Decorator: This marks the function as a LangChain tool, making it available for use by a language model agent. 
# Tools in LangChain are functions that agents can call to perform specific tasks.
# Mentioning the prompt inside as in Mongodb (https://langchain-mongodb.readthedocs.io/en/latest/langchain_mongodb/agent_toolkit/langchain_mongodb.agent_toolkit.database.MongoDBDatabase.html#langchain_mongodb.agent_toolkit.database.MongoDBDatabase.run_no_throw) only uses aggregation queries.

@tool
def query_to_database(query: str) -> str:
    """
        Execute a MongoDB **aggregation query string** against the database and return the result.
        The query string MUST be in the MongoDB shell format: 'db.collectionName.aggregate([pipeline])'.
        
        DONT USE other aggregation queries like find, findOne, etc.
        
        Example query: 
        
        **IMPORTANT TOOL USAGE RULES:**
        1.  The `query_to_database` tool ONLY accepts MongoDB **aggregation query strings**.
        2.  The query string MUST strictly follow the format: `'db.collectionName.aggregate([pipeline])'`.
        3.  Use the correct collection name (e.g., `users`, `payments`).
        4.  Use the correct field names based on the known schema (e.g., `firstName`, `lastName`, `emailId`, `createdAt`). Do NOT guess field names like `first_name`.
        5.  Use `$match` within the pipeline for filtering documents (like a WHERE clause).
        6.  Use `$project` to select specific fields.
        7.  Use `$count` to count documents.
        8.  Use `$limit` and `$sort` for those specific operations.
        9.  Do NOT attempt to use other commands like `find`, `findOne`, `countDocuments` directly in the query string.

        **QUERY EXAMPLES:**

        **1. Show all documents in a collection (e.g., `users`):**
        *User Request:* "Show all users", "List all users"
        *Tool Query:* `'db.users.aggregate([ { "$match": {} } ])'`

        **2. Show documents matching specific criteria (e.g., users with `firstName` "Rohan"):**
        *User Request:* "Find users named Rohan", "Get user Rohan's details"
        *Tool Query:* `'db.users.aggregate([ { "$match": { "firstName": "Rohan" } } ])'`

        **3. Show documents matching multiple criteria (e.g., users with `firstName` "Rohan" AND `lastName` "Gore"):**
        *User Request:* "Find user Rohan Gore"
        *Tool Query:* `'db.users.aggregate([ { "$match": { "firstName": "Rohan", "lastName": "Gore" } } ])'`
        *(Alternative using $and):* `'db.users.aggregate([ { "$match": { "$and": [ { "firstName": "Rohan" }, { "lastName": "Gore" } ] } } ])'`

        **4. Show specific fields for matching documents (e.g., `firstName` and `emailId` for user "Rohan"):**
        *User Request:* "Show Rohan's first name and email"
        *Tool Query:* `'db.users.aggregate([ { "$match": { "firstName": "Rohan" } }, { "$project": { "firstName": 1, "emailId": 1, "_id": 0 } } ])'`

        **5. Count documents matching criteria (e.g., count users named "Rohan"):**
        *User Request:* "How many users are named Rohan?"
        *Tool Query:* `'db.users.aggregate([ { "$match": { "firstName": "Rohan" } }, { "$count": "matching_users_count" } ])'`

        **6. Limit the number of results (e.g., show the first 5 users):**
        *User Request:* "Show 5 users"
        *Tool Query:* `'db.users.aggregate([ { "$match": {} }, { "$limit": 5 } ])'`

        **7. Sort results (e.g., show users sorted by `createdAt` descending):**
        *User Request:* "Show users sorted by creation date, newest first"
        *Tool Query:* `'db.users.aggregate([ { "$match": {} }, { "$sort": { "createdAt": -1 } } ])'`

        **8. Combine operations (e.g., show `emailId` of the 5 newest users named "Rohan"):**
        *User Request:* "Show the email addresses of the 5 most recent users named Rohan"
        *Tool Query:* `'db.users.aggregate([ { "$match": { "firstName": "Rohan" } }, { "$sort": { "createdAt": -1 } }, { "$limit": 5 }, { "$project": { "emailId": 1, "_id": 0 } } ])'`

        Remember to always construct the query string in the exact `db.collectionName.aggregate([pipeline])` format for the `query_to_database` tool. Use the collection schema information (like field names `firstName`, `emailId`) when formulating the pipeline stages.
"""
    
    # runs the query and if it is invalid or returns no result, gracefully handles it will return an error message.
    result = db.run_no_throw(query) 
    
    
    if not result:
        return "No result returned from the query. Please try again."
    return result

In [101]:
db.get_collection_info(["users"])

'Database name: devTinder\nCollection name: users\nSchema from a sample of documents from the collection:\n_id: ObjectId\nfirstName: String\nlastName: String\nemailId: String\npassword: String\nphotoUrl: String\nskills: Array\ncreatedAt: Timestamp\nupdatedAt: Timestamp\n__v: Number\n\n/*\n3 documents from users collection:\n[\n  {\n    "_id": {\n      "$oid": "66ece21ff3406ae729fafc0c"\n    },\n    "firstName": "Rohan",\n    "lastName": "Gore",\n    "emailId": "rg@gmail.com",\n    "password": "$2b$10$ptCy5NAP59AF5t",\n    "photoUrl": "http://dummy.com",\n    "skills": [],\n    "createdAt": {\n      "$date": "2024-09-20T02:46:55.558Z"\n    },\n    "updatedAt": {\n      "$date": "2024-09-20T02:46:55.558Z"\n    },\n    "__v": 0\n  },\n  {\n    "_id": {\n      "$oid": "66eec283bf081b9c5cb8c96e"\n    },\n    "firstName": "Vibhor",\n    "lastName": "J",\n    "emailId": "vb@gmail.com",\n    "password": "$2b$10$hMCB8xIJxKcr1y",\n    "photoUrl": "http://dummy.com",\n    "skills": [],\n    "crea

In [102]:
# Example of using the tool
query_to_database.invoke('db.users.aggregate([ { "$match": { "firstName": "Sahil" } } ])')

'[\n  {\n    "_id": {\n      "$oid": "67ce3b18d583a646a05cb04f"\n    },\n    "firstName": "Sahil",\n    "lastName": "Bhoir",\n    "emailId": "sb@gmail.com",\n    "password": "$2b$10$HagDQ/B00ra/BpR8tXTSy.ANQMuhsTSaPkwwz4UBn2TYpvXKGub2q",\n    "photoUrl": "http://dummy.com",\n    "skills": [],\n    "createdAt": {\n      "$date": "2025-03-10T01:06:32.236Z"\n    },\n    "updatedAt": {\n      "$date": "2025-03-12T01:59:29.740Z"\n    },\n    "__v": 0,\n    "isPayment": true\n  }\n]'

In [103]:
query_to_database.invoke('db.users.aggregate([ { "$match": { } } ])')

'[\n  {\n    "_id": {\n      "$oid": "66ece21ff3406ae729fafc0c"\n    },\n    "firstName": "Rohan",\n    "lastName": "Gore",\n    "emailId": "rg@gmail.com",\n    "password": "$2b$10$ptCy5NAP59AF5txOtIutV.CmoIZoqh1TK1kAaEWpIQjQSjlTwhdgi",\n    "photoUrl": "http://dummy.com",\n    "skills": [],\n    "createdAt": {\n      "$date": "2024-09-20T02:46:55.558Z"\n    },\n    "updatedAt": {\n      "$date": "2024-09-20T02:46:55.558Z"\n    },\n    "__v": 0\n  },\n  {\n    "_id": {\n      "$oid": "66eec283bf081b9c5cb8c96e"\n    },\n    "firstName": "Vibhor",\n    "lastName": "J",\n    "emailId": "vb@gmail.com",\n    "password": "$2b$10$hMCB8xIJxKcr1y5Ho9s94.jSm2/TMNZRWK0ojct5hVigojFJ7hLb.",\n    "photoUrl": "http://dummy.com",\n    "skills": [],\n    "createdAt": {\n      "$date": "2024-09-21T12:56:35.503Z"\n    },\n    "updatedAt": {\n      "$date": "2024-09-26T03:18:45.028Z"\n    },\n    "__v": 0\n  },\n  {\n    "_id": {\n      "$oid": "66f0e4794fad30bd74decb15"\n    },\n    "firstName": "Shirish

In [104]:
## Tool binding
"""
First, it binds the query_to_database tool to the language model (LLM).
This essentially gives the LLM access to the database query functionality.
"""
llm_with_tools = llm.bind_tools([query_to_database])

# Now, when the LLM is asked to show all employees, it can use the query_to_database tool to execute the query.
llm_with_tools.invoke("show all users") # it executes "select * from employees;"


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kgab', 'function': {'arguments': '{"query":"\'db.users.aggregate([ { \\"$match\\": {} } ])\'"}', 'name': 'query_to_database'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 1899, 'total_tokens': 1983, 'completion_time': 0.07, 'prompt_time': 0.246602884, 'queue_time': 0.238713853, 'total_time': 0.316602884}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6a95f949-2f65-4320-bd71-c28b22382c88-0', tool_calls=[{'name': 'query_to_database', 'args': {'query': '\'db.users.aggregate([ { "$match": {} } ])\''}, 'id': 'call_kgab', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1899, 'output_tokens': 84, 'total_tokens': 1983})